# logstar data download guide
This notebook gives you an introduction for the logstar-online-stream download tool written in python. You can find the sourcecode under: https://github.com/zalf-rdm/Logstar-online-Stream.
Using this tool, LoraWan-sensor-data can be downloaded raw, or with "cleaned".

In this example notebook we're going to install and import the required python packages, download only water_content data from all stations(patches) for the duration between 2021-01-01 and 2022-01-01.

## UI Interface

This little UI interface helps with configurating the download request. Please select starting date, ending date, the station you want to collect data for and activate the filters if you want to use them.

In [1]:
import ipywidgets as widgets

stations = [
            # weather
            "ws1_l1_rtu_BL",
            "ws2_l1_rtu_BL",
            "tbsl1_00172_BL",
            # stationary
            "tbs6a_01_180048_BL",
            "tbs6a_02_180049_BL",
            "tbs6a_03_180050_BL",
            "tbs6a_04_180051_BL",
            "tbs6a_05_180052_BL",
            "tbs6a_06_180054_BL",
            "tbs6a_07_180055_BL",
            "tbs6a_08_180056_BL",
            "tbs6a_09_180057_BL",
            "tbs6a_10_180058_BL",
            "tbs6a_11_180059_BL",
            "tbs6a_12_180060_BL",
            "tbs6a_13_180061_BL",
            "tbs6a_14_180108_BL",
            "tbs6a_15_180063_BL",
            "tbs6a_16_180068_BL",
            "tbs6a_17_180069_BL",
            "tbs6a_18_180070_BL",
            "tbs6a_19_180071_BL",
            "tbs6a_20_180072_BL",
            "tbs6a_21_180073_BL",
            "tbs6a_22_180075_BL",
            "tbs6a_23_180076_BL",
            "tbs6a_24_180078_BL",
            "tbs6a_25_180081_BL",
            "tbs6a_26_180082_BL",
            "tbs6a_27_180083_BL",
            "tbs6a_28_180084_BL",
            "tbs6a_29_180085_BL",
            "tbs6a_30_180086_BL",
            # # mobile
            "wcecst_01_BL",
            "wcecst_02_BL",
            "wcecst_03_BL",
            "wcecst_04_BL",
            "wcecst_05_BL",
            "wcecst_06_BL",
            "wcecst_07_BL",
            "wcecst_08_BL",
            "wcecst_09_BL",
            "wcecst_10_BL"
]
apikey = widgets.Text(
    value="",
    placeholder='',
    disabled=False
)

startdate = widgets.DatePicker(
    disabled=False
)

enddate = widgets.DatePicker(
    disabled=False
)
datepickers = widgets.VBox([startdate, enddate])

stations_select = widgets.SelectMultiple(
    options=stations,
    value=stations,
    #rows=10,
    description='stations:',
    disabled=False,
    rows=20,
)

# filter BC
bulkconductifity_enabled = widgets.Checkbox(
    value=False,
    description='BulkConductivityDrift Filter',
    disabled=False,
    indent=False
)
bulkconductifity_left_right_text = widgets.Text(
    value='50',
    placeholder='50',
    description='',
    disabled=False
)
bulkconductifity_between_depth_text = widgets.Text(
    value='80',
    placeholder='80',
    description='',
    disabled=False
)
bulkconductifity_max_value_text = widgets.Text(
    value='300',
    placeholder='300',
    description='',
    disabled=False
)

# JumpCheckPS
watercontentfiler_enabled = widgets.Checkbox(
    value=False,
    description='WaterContentJump Filter',
    disabled=False,
    indent=False
)

blacklistfiler_battery_voltage_enabled = widgets.Checkbox(
    value=False,
    description='Filter out battery_voltage',
    disabled=False,
    indent=False
)

blacklistfiler_signal_strength_enabled = widgets.Checkbox(
    value=False,
    description='Filter out signal_strength',
    disabled=False,
    indent=False
)

widgets.VBox(
    [
     widgets.HBox([widgets.Label("Logstar API Key:"), apikey]),
     widgets.HBox([widgets.Label("Pick a startdate:"), startdate]),
     widgets.HBox([widgets.Label("Pick an enddate:"), enddate]),
     widgets.Label("Select stations, use SHIFT, CTRL:"), 
     stations_select,
     widgets.Label("Enable bulk conductivity filter:"),
     widgets.HBox(
         [
             bulkconductifity_enabled,
             widgets.VBox(
             [
                widgets.Label("Set BULK_CONDUCTIVITY_THRESHOLD_BETWEEN_LEFT_RIGHT:"),
                bulkconductifity_left_right_text,
                widgets.Label("set BULK_CONDUCTIVITY_THRESHOLD_BETWEEN_DEPTH:"),
                bulkconductifity_between_depth_text,
                widgets.Label("set BULK_CONDUCTIVITY_THRESHOLD_MAX_VALUE:"),
                bulkconductifity_max_value_text,
             ]),
         ]
     ),
     widgets.Label("Enable water content filter:"),
     watercontentfiler_enabled,       
     widgets.Label("Filter out Meta Columns:"),
     widgets.HBox(
         [
            blacklistfiler_battery_voltage_enabled,
            blacklistfiler_signal_strength_enabled
         ]),
     ]
)

before downloading the data we have to do some preparations. first of all install the logstar-online-stream python package with all requirements via:

In [ ]:
#! python -m pip install git+https://github.com/zalf-rdm/Logstar-online-Stream.git
#! mkdir data/

# Start Download Process

this will delete all files in logs and data folder, so if you want to keep files from there, backup them in advance.

In [ ]:
import logging
import json
import os
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)

import logstar_stream.logstar as logstar
import logstar_stream.processing_steps.ProcessingStep as ps

# load mapping file to translate sensor name to patch name and meassurement acronyms to names.
sensor_mapping = ""
with open("patchcrop-sensor-mapping.json") as jsonfile:
        sensor_mapping = json.load(jsonfile)


if None in [startdate.value, enddate.value, apikey.value, stations_select.value]: raise ValueError("one required filed is not filled out correctly, check startdate, enddate, stations and apikey")
conf = {
    "apikey": apikey.value, # logstar api key
    "stationlist": list(stations_select.value), # list of stations to process
    "geodata": True, # Returns longitude and latitude of the station as well as a comment (not implemented, i guess)
    "datetime": 0, #  Date and time format in the channel list: integer: 0/1
                   #  0 (default): „dateTime“: „2020-04-01 00:00:00“
                   #  1: „date“: „2020-04-01“, „time“: „00:00:00“ 
                   # USE 1 AS IT IS EXPECTED IN PS
    "startdate": startdate.value.strftime("%Y-%m-%d"), # Day from which the data should be retrieved in the format: YYYY-MM-DD
    "enddate": enddate.value.strftime("%Y-%m-%d") # Day to which the data is to be retrieved in the format: YYYY-MM-DD
}
    
BULK_CONDUCTIVITY_THRESHOLD_BETWEEN_LEFT_RIGHT = bulkconductifity_left_right_text.value if bulkconductifity_left_right_text.value else 50 # threshold to filter out. If left or right is this value above the other. The higher one gets filtered out
BULK_CONDUCTIVITY_THRESHOLD_BETWEEN_DEPTH = bulkconductifity_between_depth_text.value if bulkconductifity_between_depth_text.value  else 80 # if 60cm is higher than 30cm by at least this value or 90cm is higher than 60cm by this value 60cm or 90cm value got filtered out
BULK_CONDUCTIVITY_THRESHOLD_MAX_VALUE  = bulkconductifity_max_value_text.value if bulkconductifity_max_value_text.value else 300 # Maximum allow BC if above it will be filtered out 
processing_steps = []

if watercontentfiler_enabled.value: processing_steps.append(ps.load_class(["JumpCheckPS"]))
if bulkconductifity_enabled.value: processing_steps.append(
    ps.load_class(
        [
            "BulkConductivityDriftPS",  # step name
            f"treshold_left_to_right={BULK_CONDUCTIVITY_THRESHOLD_BETWEEN_LEFT_RIGHT}", # arguments
            f"threshold_between_depth={BULK_CONDUCTIVITY_THRESHOLD_BETWEEN_DEPTH}",
            f"threshold_max_value={BULK_CONDUCTIVITY_THRESHOLD_MAX_VALUE}"
        ]
    )
)
bv="battery_voltage" if blacklistfiler_battery_voltage_enabled.value else ""
ss="signal_strength" if blacklistfiler_signal_strength_enabled.value else ""
processing_steps.append(
    ps.load_class(
        [
            "BlacklistFilterColumnsPS",
            f"columns={bv} {ss}"
        ]
    )
)
df_dict = logstar.manage_dl_db( conf,                              # configuration
                                processing_steps=processing_steps, # loaded processing steps
                                sensor_mapping=sensor_mapping,     # translation file
                                csv_folder="data/sensor_data/")                # folder to write csv files to